# Bayesian A/B-Testing

In [ ]:
# !which python
# !pip install nbformat
# !pip install kaleido

# !makedir images

In [ ]:
import sys
sys.path.append('../')

from typing import Dict, List, Any, Union

import os
import numpy as np
import pandas as pd

from tqdm import tqdm

from scipy import stats
from scipy.stats import beta, gamma

# import util functions
from utils.campaign import Campaign
from utils.hypothesis_test import Hypothesis_AB_Test
from utils.bayesian_test import Bayesian_AB_Test

# import visualisation / plotting
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.animation as manimation
from matplotlib.colors import rgb2hex

from utils.graph import visualisation # conda install -n python3 -c conda-forge colorlover
import plotly
import plotly.graph_objects as go
# Init visualisation tool
plot = visualisation(renderer="vscode") # vscode | iframe for browsers

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.width', 10000)

tqdm.pandas()

# Parameters

In [ ]:
# Define Campaign Parameters
# Click-Through-Rate
CTR_A = 0.10
CTR_B = 0.12
# Cost-per-Mill average value
CPM_MEAN_A = 11
CPM_MEAN_B = 19

# ---------------------------------------------------------
# Campaign simulation pararmeters
N_PERIODS = 2000+1
N_IMPR_PER_PERIOD = 5
SEED = 10 # 10 # both similar


# ---------------------------------------------------------
# For single evalution - number of impressons
N_IMPR = 1000 # exact # of impressions according to the hypothesis test


# Config for evaluation
config ={'metrics':
            {'ks': True, # calc. Kolmorogorov-Smirnov
             'ws': True, # calc. Wasserstein
             'jsd': False, # calc. Jensen-Shannon-Divergence
             'n_samples': 1000} # number of samples
         }

# Plotting
WIDTH_SAVE, HEIGHT_SAVE = 1200, 400

# Images and videos
DO_MAKE_VIDEOS = True

# create image folder, if not exists
IMAGE_FOLDER = "../../images"
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)

VIDEO_FOLDER = "../../video"
if not os.path.exists(VIDEO_FOLDER):
    os.makedirs(IMAGE_FOLDER)

# Statistics for the single evaluation

In [ ]:
# Calc. number of click after N_IMPR impressions
n_clicks_a = int(CTR_A*N_IMPR)
n_clicks_b = int(CTR_B*N_IMPR)

# Calc. cost after N_IMPR impressions
cost_a = N_IMPR*CPM_MEAN_A/1000
cost_b = N_IMPR*CPM_MEAN_B/1000

print(f'Observations:')
print(f'Impressions: {N_IMPR} - clicks A/B: {n_clicks_a} / {n_clicks_b} - cost A/B: {cost_a} / {cost_b}\n')

print(f'Average statistics (without uncertainty):')
print(f'Click-Through-Rate (CTR) A/B: {100*n_clicks_a/N_IMPR:.1f}% / {100*n_clicks_b/N_IMPR:.1f}%')
print(f'Cost-per-Click (CpC) A/B: {CPM_MEAN_A/n_clicks_a:.3f} / {CPM_MEAN_B/n_clicks_b:.3f}\n')

<hr>

# Hypothesis testing - single evaluation

In [ ]:
hypo = Hypothesis_AB_Test()

In [ ]:
n_samples_required = hypo.calc_sample_size(CTR_A, CTR_B)
print(f'Required sample size: {n_samples_required}')

# Chi-squared test
chi2, p, dof, ex = hypo.chi2_test( n_clicks_a, N_IMPR-n_clicks_a, n_clicks_b, N_IMPR-n_clicks_b )
print(f'chi2: {chi2:.3f}')
print(f'p-value: {p:.3f}')

<hr>

# Bayesian Testing - single evaluation

In [ ]:
bayes = Bayesian_AB_Test()

### Probabilistic & Loss Analysis - on static data

In [ ]:
# Click-Through-Rate (CTR) - Beta distribution
# calc. parameters
alpha_a, beta_a = n_clicks_a+1, N_IMPR-n_clicks_a
alpha_b, beta_b = n_clicks_b+1, N_IMPR-n_clicks_b
print(f"alpha_A: {alpha_a} - beta_a: {beta_a} - alpha_b: {alpha_b} - beta_b: {beta_b}")

# Calc. probabilities and losses
print(f'Performance metric: Click-Through-Rate (CTR)')
P, (loss_a, loss_b) = bayes.p_ab_loss( [beta(alpha_a,beta_a), beta(alpha_b,beta_b)], thr=1, n_samples=10000 )
print(f'Probability of winner A/B, p(ratio>1) = {P[0]*100:.1f}% / {P[1]*100:.1f}% (sampled)')
# P = bayes.p_ba_beta(alpha_a, beta_a, alpha_b, beta_b)
# print(f'Probability of winner A/B, p(ratio>1) = {(1-P)*100:.1f}% / {P*100:.1f}% (analytically)')
# loss_a, loss_b = bayes.loss(beta(alpha_a,beta_a), beta(alpha_b, beta_b), f_min=0, f_max=1, N=100)
# loss_a, loss_b = bayes.loss(beta(alpha_a,beta_a), beta(alpha_b, beta_b), N=100)
print(f'Loss CTR A/B: {loss_a:.2f} / {loss_b:.2f} (integration)')
# loss_a, loss_b = bayes.loss_beta(alpha_a, beta_a, alpha_b, beta_b)
# print(f'Loss CTR A/B: {100*loss_a:.2f} / {100*loss_b:.2f} (analytically)\n')

# Cost-per-Click (CpC) - Gamma distribution
# calc. parameters
cost_a, scale_a = N_IMPR*CPM_MEAN_A/1000, 1/n_clicks_a
cost_b, scale_b = N_IMPR*CPM_MEAN_B/1000, 1/n_clicks_b
a_a, a_b = cost_a+1, cost_b+1
# Calc. probabilities and losses
print(f'Performance metric: Cost-per-Click (CpC)')
P, (loss_a, loss_b) = bayes.p_ab_loss( [gamma(a=a_a, scale=scale_a), gamma(a=a_b, scale=scale_b)], best='min', thr=1, n_samples=10000 )
print(f'Probability of cheapest click (A/B), p(ratio>1) = {P[0]*100:.1f}% / {P[1]*100:.1f}% (sampled)')
# P = bayes.p_ba( rv_a=gamma(a=a_a, scale=scale_a), rv_b=gamma(a=a_b, scale=scale_b), n_samples=10000 )
# print(f'Probability of cheapest click (A/B), p(ratio>1) = {P*100:.1f}% / {(1-P)*100:.1f}% (sampled) - correct')
# loss_b, loss_a = bayes.loss(gamma(a=a_a, scale=scale_a), gamma(a=a_b, scale=scale_b))
print(f'Loss CpC A/B: {loss_a:.2f} / {loss_b:.2f}')

## Bayesian Metrics

### Probability of winner - Click-Through-Rate (CTR)

In [ ]:
# plot CTR Beta distributions
# scan lift range
thr = 1.0
n_samples = 1_000_000
rv_a, rv_b = beta(alpha_a, beta_a), beta(alpha_b, beta_b)

samples_a = rv_a.rvs(size=n_samples)
samples_b = rv_b.rvs(size=n_samples)
ratio = samples_b / samples_a
hist, bins = np.histogram(ratio, bins=1000)
id1, id2 = bins<=thr, bins>thr

p_ratio_A = np.sum(hist[id1[:-1]]) / n_samples
p_ratio_B = np.sum(hist[id2[:-1]]) / n_samples
print(f'Probability of A as winner, p(ratio>1) = {p_ratio_A*100:.1f}%')
print(f'Probability of B as winner, p(ratio>1) = {p_ratio_B*100:.1f}%')

# loss calculation
P, (loss_a, loss_b) = bayes.p_ab_loss(rvs=[rv_a, rv_b], n_samples=10_000)
print(f'P CTR A/B: {100*P[0]:.2f}% / {100*P[1]:.2f}%')
print(f'Loss CTR A/B: {loss_a:.2f} / {loss_b:.2f}')

# ---------------------------------------------------------------------------------------------
x2 = np.linspace(0, 0.3, 1001)
p_data = [plot.plot(x=x2, y=beta.pdf(x2, alpha_a, beta_a), color=0, opacity=0.7, name='A', showlegend=True),
          plot.plot(x=x2, y=beta.pdf(x2, alpha_b, beta_b), color=1, opacity=0.7, name='B', showlegend=True)]
layout = plot.layout(title=f'', x_label='CTR', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

p_data = [plot.plot(x=bins[id1], y=hist[id1[:-1]]/sum(hist), color=0, opacity=0.7, name='A', showlegend=True),
          plot.plot(x=bins[id2], y=hist[id2[:-1]]/sum(hist), color=1, opacity=0.7, name='B', showlegend=True) ]
layout = plot.layout(title="", x_label='lift ratio', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

p_data = [ plot.plot(x=bins[id1], y=1-hist.cumsum()[id1[:-1]]/sum(hist), color=0, opacity=0.7, name='A', showlegend=True),
           plot.plot(x=bins[id2], y=1-hist.cumsum()[id2[:-1]]/sum(hist), color=1, opacity=0.7, name='B', showlegend=True) ]
layout = plot.layout(title="", x_label='lift ratio', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

### Probability of winner - Cost-per-Click (CpC)

In [ ]:
# plot CpC Gammz distributions
# scan lift range
thr = 1.0
n_samples = 1_000_000

rv_a, rv_b = gamma(a=a_a, scale=scale_a), gamma(a=a_b, scale=scale_b)
samples_a = rv_a.rvs(size=n_samples)
samples_b = rv_b.rvs(size=n_samples)

ratio = samples_a / samples_b # note: inverse ratio
hist, bins = np.histogram(ratio, bins=1000)
id1, id2 = bins<=thr, bins>thr

p_ratio_A = np.sum(hist[id1[:-1]]) / n_samples
p_ratio_B = np.sum(hist[id2[:-1]]) / n_samples
print(f'Probability of A as winner, p(ratio>1) = {p_ratio_A*100:.1f}%')
print(f'Probability of B as winner, p(ratio>1) = {p_ratio_B*100:.1f}%')

# loss calculation
P, (loss_a, loss_b) = bayes.p_ab_loss(rvs=[rv_a, rv_b], best='min', n_samples=10_000)
print(f'P CpC A/B: {100*P[0]:.2f}% / {100*P[1]:.2f}%')
print(f'Loss CpC A/B: {loss_a:.2f} / {loss_b:.2f}')

# ---------------------------------------------------------------------------------------------
x2 = np.linspace(0, 1, 1001)
p_data = [plot.plot(x=x2, y=gamma.pdf(x2, a=a_a, scale=scale_a), color=0, opacity=0.7, name='A', showlegend=True),
          plot.plot(x=x2, y=gamma.pdf(x2, a=a_b, scale=scale_b), color=1, opacity=0.7, name='B', showlegend=True)]
layout = plot.layout(title=f'', x_label='CpC', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

p_data = [plot.plot(x=bins[id1], y=hist[id1[:-1]]/sum(hist), color=0, opacity=0.7, name='A', showlegend=True),
          plot.plot(x=bins[id2], y=hist[id2[:-1]]/sum(hist), color=1, opacity=0.7, name='B', showlegend=True)]
layout = plot.layout(title="", x_label='price ratio', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

p_data = [plot.plot(x=bins[id1], y=hist.cumsum()[id1[:-1]]/sum(hist), color=0, opacity=0.7, name='A', showlegend=False),
          plot.plot(x=bins[id2], y=hist.cumsum()[id2[:-1]]/sum(hist), color=1, opacity=0.7, name='B', showlegend=False)]
layout = plot.layout(title="", x_label='price ratio', y_label='p', theme='dark', width=1200, height=400)
layout['xaxis']['range'] = [0.2, 4]
fig = go.Figure(data=p_data, layout=layout).show()

<hr>

# Power Analysis

Calculate the required number of impressions before a certain p-value or probability of a winer is achieved.
This always done for hypothesis testing, as we can only declare significance if the sufficient number of samples is observded.

Comparing to Bayesian testing, it becoems quite clear how the number of samples required is much lower for Bayesian testing

In [ ]:
# define
impr_list = np.arange(1, 100_000, 200)
n_impr = len(impr_list)
lift_list = [10, 5, 3, 2]
CTR_A = 0.10

PA = {'n_samples_required': {'ctr':{}},
      'hypo':{'ctr':{}},
      'bayes':{'ctr':{}}}
for lift in lift_list:
    PA['hypo']['ctr'][lift] = []
    PA['bayes']['ctr'][lift] = []

    # calc. CTR_B
    CTR_B = CTR_A * (1+lift/100)

    # hypothesis testing - number of samples required
    PA['n_samples_required']['ctr'][lift] = hypo.calc_sample_size(CTR_A, CTR_B)

    for impr in impr_list:
        # clicks
        clicks_a = int(impr * CTR_A)
        clicks_b = int(impr * CTR_B)

        # hypothesis testing
        PA['hypo']['ctr'][lift] += [ 1 - hypo.chi2_test(clicks_a, impr, clicks_b, impr)[1] ]

        # calc. prob. of B > A
        P, _ = bayes.p_ab_loss( [beta(clicks_a+1, impr-clicks_a+1), beta(clicks_b+1, impr-clicks_b+1)], best='max', n_samples=10000 )
        PA['bayes']['ctr'][lift] += [ P[1] ]

        # Power Analysis for Bayesian Test
        samples_A = beta(clicks_a+1, impr-clicks_a+1).rvs(size=1000)
        samples_B = beta(clicks_b+1, impr-clicks_b+1).rvs(size=1000)
        P_ = np.sum(samples_B > samples_A)

PA['n_samples_required']

In [ ]:
METRIC = 'ctr'

# plot
p_data = []
for i, (k, v) in enumerate( PA['hypo'][METRIC].items() ):
    p_data += [plot.plot(x=impr_list, y=v, color=i, opacity=0.7, name=f'Lift: {k}%', showlegend=True)]
p_data += [ plot.plot(x=impr_list, y=n_impr*[0.95], color=0, opacity=1, fill=None, linewidth=3, name='>0.95', showlegend=True) ]
layout = plot.layout(title=f'Decision Analysis for {METRIC} = {CTR_A*100}%', x_label='impressions [#]', y_label='1 - p-value', theme='dark', width=800, height=500) # width=1200, height=400)
layout['yaxis']['range'] = [0, 1]
fig = go.Figure(data=p_data, layout=layout).show()

# plot
p_data = []
for i, (k, v) in enumerate( PA['bayes'][METRIC].items() ):
    p_data += [plot.plot(x=impr_list, y=v, color=i, opacity=0.7, name=f'Lift: {k}%', showlegend=True)]
p_data += [ plot.plot(x=impr_list, y=n_impr*[0.95], color=0, opacity=1, fill=None, linewidth=3, name='>0.95', showlegend=True) ]    
layout = plot.layout(title=f'Decision Analysis for {METRIC} = {CTR_A*100}%', x_label='impressions [#]', y_label='winner probability', theme='dark', width=800, height=500) # width=1200, height=400)
layout['yaxis']['range'] = [0.5, 1]
fig = go.Figure(data=p_data, layout=layout).show()

<hr>

# Simulate campaign

### Create synth. campaign data

In [ ]:
# Campaign parameters
params = {'A': {'ctr': CTR_A, 'cpm':CPM_MEAN_A}, 'B': {'ctr': CTR_B, 'cpm':CPM_MEAN_B}, 'n_impr_per_period':N_IMPR_PER_PERIOD}
campaign = Campaign(params, random_seed=SEED)
df = campaign.create(n_periods=N_PERIODS)
df = campaign.agg_stats(df)

# total amount of impressions for A1, A2, B1, B2 and A & B
n_impressions_a1, n_impressions_a2 = df.n_impr_a1.sum(), df.n_impr_a2.sum()
n_impressions_b1, n_impressions_b2 = df.n_impr_b1.sum(), df.n_impr_b2.sum()
n_impressions_a, n_impressions_b = df.n_impr_a.sum(), df.n_impr_b.sum()

df.head(5)
print(f'Impressions')
print(f'A (total): {n_impressions_a} - A1/A2: {n_impressions_a1}/{n_impressions_a2}')
print(f'B (total): {n_impressions_b} - B1/B2: {n_impressions_b1}/{n_impressions_b2}')

#### Visualise campaign over time

In [ ]:
# Impressions / Clicks over time
p_data = [ plot.plot(x=df.t, y=df.acc_impr_a1, color=0, opacity=0.2, name='impr. A', showlegend=True),
           plot.plot(x=df.t, y=df.acc_clicks_a1, color=0, opacity=0.7, name='clicks A', showlegend=True),
           plot.plot(x=df.t, y=df.acc_impr_b1, color=1, opacity=0.2, name='impr. B', showlegend=True),
           plot.plot(x=df.t, y=df.acc_clicks_b1, color=1, opacity=0.7, name='clicks B', showlegend=True),
           ]
layout = plot.layout(title=f'Observations - impr. & clicks', x_label='time', y_label='#', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/impr_clicks.png')

# CTR / CpC over time
p_data = [ plot.plot(x=df.t, y=df.acc_clicks_a/df.acc_impr_a, color=0, opacity=0.5, name='CTR A', showlegend=True),
           plot.plot(x=df.t, y=df.acc_clicks_b/df.acc_impr_b, color=1, opacity=0.5, name='CTR B', showlegend=True),
           plot.plot(x=df.t, y=df.acc_cost_a/df.acc_clicks_a, color=0, opacity=0.5, name='CpC A', showlegend=True),
           plot.plot(x=df.t, y=df.acc_cost_b/df.acc_clicks_b, color=1, opacity=0.5, name='CpC B', showlegend=True),
           ]
layout = plot.layout(title=f'Performance - CTR & CpC', x_label='time', y_label='', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/performance.png')


### Calc. AB test stats on synth. campaign data

In [ ]:
# HYPO
df = hypo.transform(df)

# BAYES
df = bayes.transform(df, config)

In [ ]:
df.head(10)

## Visualise

#### Analyse specific time period

In [ ]:
# Choose specific time period to look at
# T = 950
T = 2000

# Extract dataframe (row) for that time
df_T = df[df.t==T]
df_T

In [ ]:
# Click-Through-Rate - Beta distribution
print( f'Beta distributions at T:{T} - Impr A: {df_T.acc_impr_a1.values[0]} / {df_T.acc_impr_a2.values[0]} - Impr B: {df_T.acc_impr_b1.values[0]} / {df_T.acc_impr_b2.values[0]}' )
print( f'- Clicks A: {df_T.acc_clicks_a1.values[0]} / {df_T.acc_clicks_a2.values[0]} - Clicks B: {df_T.acc_clicks_b1.values[0]} / {df_T.acc_clicks_b2.values[0]}' )
print( f'- Cost A: {df_T.acc_cost_a1.values[0]:.3f} / {df_T.acc_cost_a2.values[0]:.3f} - Cost B: {df_T.acc_cost_b1.values[0]:.3f} / {df_T.acc_cost_b2.values[0]:.3f}' )

x = np.linspace(0.05, 0.15, 1000)
p_data = [plot.plot(x=x, y=beta.pdf(x, df_T.alpha_a1, df_T.beta_a1), color=0, opacity=0.7, name='A1', showlegend=True),
          plot.plot(x=x, y=beta.pdf(x, df_T.alpha_a2, df_T.beta_a2), color=0, opacity=0.7, name='A2', showlegend=True),
          plot.plot(x=x, y=beta.pdf(x, df_T.alpha_b1, df_T.beta_b1), color=1, opacity=0.7, name='B1', showlegend=True),
          plot.plot(x=x, y=beta.pdf(x, df_T.alpha_b2, df_T.beta_b2), color=1, opacity=0.7, name='B2', showlegend=True)]
layout = plot.layout(title=f'Beta distributions at T:{T}', x_label='Click-Through-Rate', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout)
fig.show()

print( f'Beta distributions at T:{T} - Impr A:{df_T.acc_impr_a.values[0]} - Impr B:{df_T.acc_impr_b.values[0]}' )
p_data = [plot.plot(x=x, y=beta.pdf(x, df_T.alpha_a, df_T.beta_a), color=0, opacity=0.7, name='A', showlegend=True),
          plot.plot(x=x, y=beta.pdf(x, df_T.alpha_b, df_T.beta_b), color=1, opacity=0.7, name='B', showlegend=True)]
layout = plot.layout(title=f'Beta distributions at T:{T}', x_label='Click-Through-Rate', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

# Cost-per-Click - Gamma distributions
print( f'Beta distributions at T:{T} - Impr A1:{df_T.acc_impr_a1.values[0]} - Impr A1:{df_T.acc_impr_a2.values[0]} - Impr B1:{df_T.acc_impr_b1.values[0]} - Impr B2:{df_T.acc_impr_b2.values[0]}' )
x = np.linspace(0, 0.3, 1000)
p_data = [plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_a1, scale=df_T.scale_a1), color=0, opacity=0.7, name='A1', showlegend=True),
          plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_a2, scale=df_T.scale_a2), color=0, opacity=0.7, name='A2', showlegend=True),
          plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_b1, scale=df_T.scale_b1), color=1, opacity=0.7, name='B1', showlegend=True),
          plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_b2, scale=df_T.scale_b2), color=1, opacity=0.7, name='B2', showlegend=True)]
layout = plot.layout(title=f'Gamma distributions at T:{T} - Impr A:{df_T.acc_impr_a1.values[0]} - Impr B:{df_T.acc_impr_a2.values[0]}', x_label='Cost-per-Click', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout)
fig.show()

p_data = [plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_a, scale=df_T.scale_a), color=0, opacity=0.7, name='A', showlegend=True),
          plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_b, scale=df_T.scale_b), color=1, opacity=0.7, name='B', showlegend=True)]
layout = plot.layout(title=f'Gamma distributions at T:{T} - Cost A:{df_T.acc_cost_a.values[0]:.3f} - Cost B:{df_T.acc_cost_b.values[0]:.3f}', x_label='Cost-per-Click', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

<hr>

# DECISION TIME

In [ ]:
# Select last event to evaluate 
# T = len(df)-2
T = 1000

df_T = df[df.t==T]
df_T

In [ ]:
# Analysis
print(f'A: Impr: {df_T.acc_impr_a.values[0]} - Clicks: {df_T.acc_clicks_a.values[0]}')
print(f'B: Impr: {df_T.acc_impr_b.values[0]} - Clicks: {df_T.acc_clicks_b.values[0]}')

## A/A test

In [ ]:
# Same/Same statistical test
print(f'A/A TEST - CTR')
print(f'AA: chi2: {df_T.chi2_A1A2_ctr.values[0]:.3f} - p-value: {100*df_T.pvalue_A1A2_ctr.values[0]:.1f}%', end='')
print(f' - ks: {df_T.P_A1A2_b_ks.values[0]:.3f}', end='') if config['metrics']['ks'] else None
print(f' - ws: {df_T.P_A1A2_b_ws.values[0]:.3f}', end='') if config['metrics']['ws'] else None
print('')
print(f'BB: chi2: {df_T.chi2_B1B2_ctr.values[0]:.3f} - p-value: {100*df_T.pvalue_B1B2_ctr.values[0]:.1f}%', end='')
print(f' - ks: {df_T.P_B1B2_b_ks.values[0]:.3f}', end='') if config['metrics']['ks'] else None
print(f' - ws: {df_T.P_B1B2_b_ws.values[0]:.3f}', end='') if config['metrics']['ws'] else None
print(f'\n')

print(f'A/A TEST - CpC')
print(f'AA: chi2: {df_T.chi2_A1A2_cpc.values[0]:.3f} - p-value: {100*df_T.pvalue_A1A2_cpc.values[0]:.1f}%', end='')
print(f' - ks: {df_T.P_A1A2_g_ks.values[0]:.3f}') if config['metrics']['ks'] else None
print(f' - ws: {df_T.P_A1A2_g_ws.values[0]:.3f}') if config['metrics']['ws'] else None
print(f'BB: chi2: {df_T.chi2_B1B2_cpc.values[0]:.3f} - p-value: {100*df_T.pvalue_B1B2_cpc.values[0]:.1f}%', end='')
print(f' - ks: {df_T.P_B1B2_g_ks.values[0]:.3f}') if config['metrics']['ks'] else None
print(f' - ws: {df_T.P_B1B2_g_ws.values[0]:.3f}') if config['metrics']['ws'] else None

#### Hypothesis testing - Chi2-test

In [ ]:
# CHI2-Test - CTR
p_data = [ plot.plot(x=df.acc_impr_a, y=df.pvalue_A1A2_ctr, color=0, opacity=0.7, name='p-value A1=A2', showlegend=True),
           plot.plot(x=df.acc_impr_b, y=df.pvalue_B1B2_ctr, color=1, opacity=0.7, name='p-value B1=B2', showlegend=True),
           plot.plot(x=df.acc_impr_b, y=df.pvalue_ctr, color=3, opacity=0.7, name='p-value A=B', showlegend=True),
           plot.plot(x=list(range(n_impressions_a)), y=n_impressions_a*[0.95], color=0, opacity=1, fill=None, linewidth=3, name='>0.95', showlegend=True),
           ]
layout = plot.layout(title=f'Chi2 test - p-value of A/A (CTR)', x_label='# impressions', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AA-test_chi2_ctr.png')

# CHI2-Test - CpC
p_data = [ plot.plot(x=df.acc_impr_a, y=df.pvalue_A1A2_cpc, color=0, opacity=0.7, name='p-value A1=A2', showlegend=True),
           plot.plot(x=df.acc_impr_b, y=df.pvalue_B1B2_cpc, color=1, opacity=0.7, name='p-value B1=B2', showlegend=True),
           plot.plot(x=df.acc_impr_b, y=df.pvalue_cpc, color=3, opacity=0.7, name='p-value A=B', showlegend=True),
           plot.plot(x=list(range(n_impressions_a)), y=n_impressions_a*[0.95], color=0, opacity=1, fill=None, linewidth=3, name='>0.95', showlegend=True),
           ]
layout = plot.layout(title=f'Chi2 test - p-value of A/A (CpC)', x_label='# impressions', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AA-test_chi2_cpc.png')

#### Bayesian A/A testing

In [ ]:
# Click-Through-Rate
x = np.linspace(0.05, 0.15, 1000)
p_data = [plot.plot(x=x, y=beta.pdf(x, df_T.alpha_a1, df_T.beta_a1), color=0, opacity=0.7, name='A1', showlegend=True),
          plot.plot(x=x, y=beta.pdf(x, df_T.alpha_a2, df_T.beta_a2), color=0, opacity=0.7, name='A2', showlegend=True),
          plot.plot(x=x, y=beta.pdf(x, df_T.alpha_b1, df_T.beta_b1), color=1, opacity=0.7, name='B1', showlegend=True),
          plot.plot(x=x, y=beta.pdf(x, df_T.alpha_b2, df_T.beta_b2), color=1, opacity=0.7, name='B2', showlegend=True)]
layout = plot.layout(title=f'Beta distributions at T:{T}', x_label='engagement rate', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AA-test_betas.png')

# Kolmogorov-Smirnov plot
if config['metrics']['ks']:
    p_data = [ plot.plot(x=df.acc_impr_a, y=df.P_A1A2_b_ks, color=0, opacity=0.5, name='P(A1=A2)', showlegend=True),
            plot.plot(x=df.acc_impr_b, y=df.P_B1B2_b_ks, color=1, opacity=0.5, name='P(B1=B2)', showlegend=True),
            plot.plot(x=df.acc_impr_a, y=df.P_AB_b_ks, color=3, opacity=0.9, name='P(AB)', showlegend=True)]
    layout = plot.layout(title=f'Kolmogorov-Smirnov A/A (CTR)', x_label='# impressions', y_label='p', theme='dark', width=1200, height=400)
    fig = go.Figure(data=p_data, layout=layout).show()
    layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
    go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AA-test_CTR_ks.png')

# Wasserstein plot
if config['metrics']['ws']:
    p_data = [ plot.plot(x=df.acc_impr_a, y=df.P_A1A2_b_ws, color=0, opacity=0.5, name='P(A1=A2)', showlegend=True),
            plot.plot(x=df.acc_impr_b, y=df.P_B1B2_b_ws, color=1, opacity=0.5, name='P(B1=B2)', showlegend=True),
            plot.plot(x=df.acc_impr_a, y=df.P_AB_b_ws, color=3, opacity=0.9, name='P(AB)', showlegend=True)]
    layout = plot.layout(title=f'Wasserstein A/A (CTR)', x_label='# impressions', y_label='p', theme='dark', width=1200, height=400)
    layout['yaxis']['range'] = [0.95, 1]
    fig = go.Figure(data=p_data, layout=layout).show()
    layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
    go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AA-test_CTR_ws.png')

In [ ]:
# Cost-per-Click
x = np.linspace(0, 0.3, 1000)
p_data = [plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_a1, scale=df_T.scale_a1), color=0, opacity=0.7, name='A1', showlegend=True),
          plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_a2, scale=df_T.scale_a2), color=0, opacity=0.7, name='A2', showlegend=True),
          plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_b1, scale=df_T.scale_b1), color=1, opacity=0.7, name='B1', showlegend=True),
          plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_b2, scale=df_T.scale_b2), color=1, opacity=0.7, name='B2', showlegend=True)]
layout = plot.layout(title=f'Gamma distributions at T:{T}', x_label='Cost-per-Click', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AA-test_CpC_gamma.png')

# Kolmogorov-Smirnov plot
if config['metrics']['ks']:
        p_data = [ plot.plot(x=df.acc_impr_a, y=df.P_A1A2_g_ks, color=0, opacity=0.5, name='P(A1=A2)', showlegend=True),
                plot.plot(x=df.acc_impr_b, y=df.P_B1B2_g_ks, color=1, opacity=0.5, name='P(B1=B2)', showlegend=True),
                plot.plot(x=df.acc_impr_a, y=df.P_AB_g_ks, color=3, opacity=0.9, name='P(AB)', showlegend=True)]
        layout = plot.layout(title=f'Kolmogorov-Smirnov A/A (CpC)', x_label='# impressions', y_label='p', theme='dark', width=1200, height=400)
        fig = go.Figure(data=p_data, layout=layout).show()
        layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
        go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AA-test_CpC_ks.png')

if config['metrics']['ws']:
    p_data = [ plot.plot(x=df.acc_impr_a, y=df.P_A1A2_g_ws, color=0, opacity=0.5, name='P(A1=A2)', showlegend=True),
               plot.plot(x=df.acc_impr_b, y=df.P_B1B2_g_ws, color=1, opacity=0.5, name='P(B1=B2)', showlegend=True),
               plot.plot(x=df.acc_impr_a, y=df.P_AB_g_ws, color=3, opacity=0.9, name='P(AB)', showlegend=True)]
    layout = plot.layout(title=f'Wasserstein A/A (CpC)', x_label='# impressions', y_label='p', theme='dark', width=1200, height=400)
    layout['yaxis']['range'] = [0.95, 1]
    fig = go.Figure(data=p_data, layout=layout).show()
    layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
    go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AA-test_CpC_ws.png')

## A/B-Test - Decision Probability

In [ ]:
print(f'DECISION METRICS - AB')
print(f'A: Impr: {df_T.acc_impr_a.values[0]} - Clicks: {df_T.acc_clicks_a.values[0]} - Cost: {df_T.acc_cost_a.values[0]:.2f}')
print(f'B: Impr: {df_T.acc_impr_b.values[0]} - Clicks: {df_T.acc_clicks_b.values[0]} - Cost: {df_T.acc_cost_b.values[0]:.2f}\n')

# Decision metrics
print(f'DECISION METRICS - AB - CTR')
print(f'chi2: {df_T.chi2_ctr.values[0]:.3f} - p-value: {100*df_T.pvalue_ctr.values[0]:.1f}%', end='')
print(f' - ks: {df_T.P_AB_b_ks.values[0]:.3f}', end='') if config['metrics']['ks'] else None
print(f' - ws: {df_T.P_AB_b_ws.values[0]:.3f}', end='') if config['metrics']['ws'] else None
print('')
print(f'CTR: P(A>B): {100*df_T.P_AB_b.values[0]:.1f}% / {100*df_T.P_BA_b.values[0]:.1f}% - loss (A/B): {df_T.loss_ctr_a.values[0]:.5f} / {df_T.loss_ctr_b.values[0]:.5f}\n')

print(f'DECISION METRICS - AB - CpC')
# print(f'chi2: {df_T.chi2_cpc.values[0]:.3f} - p-value: {100*df_T.pvalue_cpc.values[0]:.1f}%  --  ks: {df_T.P_AB_g_ks.values[0]:.3f} - ws: {df_T.P_AB_g_ws.values[0]:.3f}')
print(f'chi2: {df_T.chi2_cpc.values[0]:.3f} - p-value: {100*df_T.pvalue_cpc.values[0]:.1f}%', end='')
print(f' - ks: {df_T.P_AB_g_ks.values[0]:.3f}', end='') if config['metrics']['ks'] else None
print(f' - ws: {df_T.P_AB_g_ws.values[0]:.3f}', end='') if config['metrics']['ws'] else None
print('')
print(f'CpC p(A>B) {100*df_T.P_AB_g.values[0]:.1f}% / {100*df_T.P_BA_g.values[0]:.1f}% - loss (A/B): {df_T.loss_cpc_a.values[0]:.5f} / {df_T.loss_cpc_b.values[0]:.5f}')

#### Hypothesis testing

In [ ]:
# CHI2 - CTR
p_data = [ plot.plot(x=df.acc_impr_b, y=df.pvalue_ctr, color=1, opacity=0.7, name='p-value', showlegend=True),
           plot.plot(x=list(range(n_impressions_a)), y=n_impressions_a*[0.05], color=0, opacity=1, fill=None, linewidth=3, name='<0.05', showlegend=True),
           ]
layout = plot.layout(title=f'Chi2-test - p-value (CTR)', x_label='# impressions', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AB-test_chi2_pvalue_ctr.png')

# CHI2 - CpC
p_data = [ plot.plot(x=df.acc_impr_b, y=df.pvalue_cpc, color=1, opacity=0.7, name='p-value', showlegend=True),
           plot.plot(x=list(range(n_impressions_a)), y=n_impressions_a*[0.05], color=0, opacity=1, fill=None, linewidth=3, name='<0.05', showlegend=True),
           ]
layout = plot.layout(title=f'Chi2-test - p-value (CpC)', x_label='# impressions', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AB-test_chi2_pvalue_cpc.png')

#### Bayesian testing

In [ ]:

x = np.linspace(0.05, 0.15, 1001)
p_data = [plot.plot(x=x, y=beta.pdf(x, df_T.alpha_a, df_T.beta_a), color=0, opacity=0.7, name='CTR A', showlegend=True),
          plot.plot(x=x, y=beta.pdf(x, df_T.alpha_b, df_T.beta_b), color=1, opacity=0.7, name='CTR B', showlegend=True)]
layout = plot.layout(title=f'Performance distributions at T:{T}', x_label='engagement rate', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AB-test_CTR_betas.png')

x = np.linspace(0, 0.3, 1001)
p_data = [plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_a, scale=df_T.scale_a), color=0, opacity=0.7, name='CpC A', showlegend=True),
          plot.plot(x=x, y=gamma.pdf(x, a=df_T.a_b, scale=df_T.scale_b), color=1, opacity=0.7, name='CpC B', showlegend=True)]
layout = plot.layout(title=f'Performance distributions at T:{T}', x_label='engagement rate', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AB-test_CpC_gammas.png')


# BAYES - CTR
p_data = [ plot.plot(x=list(range(n_impressions_a)), y=n_impressions_a*[0.95], color=0, opacity=1, fill=None, linewidth=3, name='>0.95', showlegend=True),
           plot.plot(x=df.acc_impr_a, y=df.P_AB_b, color=0, opacity=0.7, name='P(A>B)', showlegend=True),
           plot.plot(x=df.acc_impr_b, y=df.P_BA_b, color=1, opacity=0.7, name='P(B>A)', showlegend=True),
           ]
layout = plot.layout(title=f'Probability of P(B>A), P(A>B) - CTR', x_label='# impressions', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AB-test_CTR_prob.png')

p_data = [ plot.plot(x=df.acc_impr_a, y=df.loss_ctr_a, color=0, opacity=0.5, name='loss A', showlegend=True),
           plot.plot(x=df.acc_impr_b, y=df.loss_ctr_b, color=1, opacity=0.8, name='loss B', showlegend=True),
           ]
layout = plot.layout(title=f'Loss - CTR', x_label='# impressions', y_label='loss', theme='dark', width=1200, height=400)
layout['yaxis']['range'] = [0, 0.05]
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AB-test_CTR_loss.png')

# BAYES - CpC
p_data = [ plot.plot(x=list(range(n_impressions_a)), y=n_impressions_a*[0.95], color=0, opacity=1, fill=None, linewidth=3, name='>0.95', showlegend=True),
           plot.plot(x=df.acc_impr_a, y=df.P_AB_g, color=0, opacity=0.7, name='P(B<A)', showlegend=True),
           plot.plot(x=df.acc_impr_b, y=df.P_BA_g, color=1, opacity=0.7, name='P(A<B)', showlegend=True),
           ]
layout = plot.layout(title=f'Probability of P(A<B), P(B<A) - CpC', x_label='# impressions', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AB-test_CpC_prob.png')

p_data = [ plot.plot(x=df.acc_impr_a, y=df.loss_cpc_a, color=0, opacity=0.5, name='loss A', showlegend=True),
           plot.plot(x=df.acc_impr_b, y=df.loss_cpc_b, color=1, opacity=0.8, name='loss B', showlegend=True),
           ]
layout = plot.layout(title=f'Loss - CpC', x_label='# impressions', y_label='loss', theme='dark', width=1200, height=400)
# layout['yaxis']['range'] = [0, 0.05]
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image(f'{IMAGE_FOLDER}/AB-test_CpC_loss.png')

### Video

In [ ]:
# conda install -c conda-forge ffmpeg
if DO_MAKE_VIDEOS:

    N_STEPS = 500
    x = np.linspace(0, 0.5, 50000)

    def make_video(framerate=10, xlabel='', n_versions=4, colormap=['#ff4444', '#ff4444', '#ff44ff', '#ff44ff'], txt_pos=0.4):
        FFMpegWriter = manimation.writers['ffmpeg']
        metadata = dict(title='Movie Test', artist='Matplotlib', comment='')
        writer = FFMpegWriter(fps=framerate, metadata=metadata)

        # fig = plt.figure
        fig = plt.figure(figsize=(12,6), dpi=100, facecolor='#202020', edgecolor='#202020')
        plt.margins(10)

        ax = plt.gca()
        fig.patch.set_facecolor('#202020')
        ax.set_facecolor('#202020')
        ax.set_xlabel(xlabel)
        ax.set_ylabel('#')
        ax.spines['bottom'].set_color('#AAAAAA')
        ax.spines['top'].set_color('#AAAAAA')
        ax.spines['left'].set_color('#AAAAAA')
        ax.spines['right'].set_color('#AAAAAA')
        ax.xaxis.label.set_color('#AAAAAA')
        ax.tick_params(axis='x', colors='#AAAAAA')

        # colorsteps = math.floor(255/n_versions)
        # colormap = [rgb2hex(np.array([255,colorsteps*i,colorsteps*i])/255) for i in range(n_versions)]
        plts = [plt.plot([], [], colormap[i], linewidth=3)[0] for i in range(n_versions)]
        plt.xlim([0, 0.25])
        plt.ylim([0, 100])
        txt_time = plt.figtext(0.7, txt_pos, "", fontsize=12, color='#AAAAAA')
        ax.get_yaxis().set_visible(False)

        return writer, fig, plt, plts, txt_time


    # A/A-test - on campaign CTR data...
    writer, fig, plt, plts, txt_time = make_video(xlabel='Click-Through-Rate (AA-test)', txt_pos=0.35)
    with writer.saving(fig, f'{VIDEO_FOLDER}/AA-test_CTR.mp4', 100):
        for t in tqdm(range(N_STEPS+1)):
            plts[0].set_data(x, stats.beta.pdf(x, df.loc[t,'alpha_a1'], df.loc[t,'beta_a1']))
            plts[1].set_data(x, stats.beta.pdf(x, df.loc[t,'alpha_a2'], df.loc[t,'beta_a2']))
            plts[2].set_data(x, stats.beta.pdf(x, df.loc[t,'alpha_b1'], df.loc[t,'beta_b1']))
            plts[3].set_data(x, stats.beta.pdf(x, df.loc[t,'alpha_b2'], df.loc[t,'beta_b2']))
            
            txt = ' time: {}\n\n A1 / A2\n impr.: {} / {}\n clicks: {} / {}\n'.format(t, df.loc[t,'acc_impr_a1'], df.loc[t,'acc_impr_a2'], df.loc[t,'acc_clicks_a1'], df.loc[t,'acc_clicks_a2'])
            txt += ' ks: {:.3f} '.format(df.loc[t,'P_A1A2_b_ks']) if config['metrics']['ks'] else ''
            txt += ' ws: {:.3f} '.format(df.loc[t,'P_A1A2_b_ws']) if config['metrics']['ws'] else ''
            txt += '\n\n'
            txt += ' B1 / B2\n impr.: {} / {}\n clicks: {} / {}\n'.format(df.loc[t,'acc_impr_b1'], df.loc[t,'acc_impr_b2'], df.loc[t,'acc_clicks_b1'], df.loc[t,'acc_clicks_b2'])
            txt += ' ks: {:.3f} '.format(df.loc[t,'P_B1B2_b_ks']) if config['metrics']['ks'] else ''
            txt += ' ws: {:.3f}'.format(df.loc[t,'P_B1B2_b_ws']) if config['metrics']['ws'] else ''
            txt_time.set_text(txt)
            writer.grab_frame(facecolor=fig.get_facecolor(), edgecolor='none')
    plt.close()

    # A/A-test - on campaign CpC data...
    writer, fig, plt, plts, txt_time = make_video(xlabel='Cost-per-Click (AA-test)', txt_pos=0.35)
    plt.xlim([0, 0.5])
    plt.ylim([0, 20])
    with writer.saving(fig, f'{VIDEO_FOLDER}/AA-test_CpC.mp4', 100):
        for t in tqdm(range(N_STEPS+1)):
            plts[0].set_data(x, stats.gamma.pdf(x, a=df.loc[t,'a_a1'], scale=df.loc[t,'scale_a1']))
            plts[1].set_data(x, stats.gamma.pdf(x, a=df.loc[t,'a_a2'], scale=df.loc[t,'scale_a2']))
            plts[2].set_data(x, stats.gamma.pdf(x, a=df.loc[t,'a_b1'], scale=df.loc[t,'scale_b1']))
            plts[3].set_data(x, stats.gamma.pdf(x, a=df.loc[t,'a_b2'], scale=df.loc[t,'scale_b2']))
            
            txt = ' time: {}\n\n A1 / A2\n clicks.: {} / {}\n cost: {:.2f} / {:.2f}\n'.format(t, df.loc[t,'acc_clicks_a1'], df.loc[t,'acc_clicks_a2'], df.loc[t,'acc_cost_a1'], df.loc[t,'acc_cost_a2'])
            txt += ' ks: {:.3f}'.format(df.loc[t,'P_A1A2_g_ks']) if config['metrics']['ks'] else ''
            txt += ' ws: {:.3f}'.format(df.loc[t,'P_A1A2_g_ws']) if config['metrics']['ws'] else ''
            txt += '\n\n'
            txt += ' B1 / B2\n clicks.: {} / {}\n cost: {:.2f} / {:.2f}\n'.format(df.loc[t,'acc_clicks_b1'], df.loc[t,'acc_clicks_b2'], df.loc[t,'acc_cost_b1'], df.loc[t,'acc_cost_b2'])
            txt += ' ks: {:.3f}'.format(df.loc[t,'P_B1B2_g_ks']) if config['metrics']['ks'] else ''
            txt += ' ws: {:.3f}'.format(df.loc[t,'P_B1B2_g_ws']) if config['metrics']['ws'] else ''
            txt_time.set_text(txt)
            writer.grab_frame(facecolor=fig.get_facecolor(), edgecolor='none')
    plt.close()


    # A/B-test - on campaign CTR data...
    writer, fig, plt, plts, txt_time = make_video(xlabel='Click-Through-Rate (AB-test)', n_versions=2, colormap=['#ff4444','#ff44ff'], txt_pos=0.4)
    plt.ylim([0, 70])
    with writer.saving(fig, f'{VIDEO_FOLDER}/AB-test_CTR.mp4', 100):
        for t in tqdm(range(N_STEPS+1)):
            plts[0].set_data(x, stats.beta.pdf(x, df.loc[t,'alpha_a'], df.loc[t,'beta_a']))
            plts[1].set_data(x, stats.beta.pdf(x, df.loc[t,'alpha_b'], df.loc[t,'beta_b']))

            txt = ' time: {}\n\n A / B\n impr.: {} / {}\n clicks: {} / {}\n\n'.format(t, df.loc[t,'acc_impr_a'], df.loc[t,'acc_impr_b'], df.loc[t,'acc_clicks_a'], df.loc[t,'acc_clicks_b'])
            txt += ' p-value: {:.1f}%\n\n'.format(100*df.loc[t,'pvalue_ctr'])
            txt += ' p(a>b): {:.1f}% / {:.1f}%\n'.format(100*df.loc[t,'P_AB_b'], 100*df.loc[t,'P_BA_b'])
            txt += ' loss: {:.3f} / {:.3f} \n\n'.format(df.loc[t,'loss_ctr_a'], df.loc[t,'loss_ctr_b'])
            txt_time.set_text(txt)

            # txt_time.set_text( " time: {}\n\n impr.: {}\n clicks: {}".format(t, df.loc[t,'acc_impr_a']+df.loc[t,'acc_impr_b'], df.loc[t,'acc_clicks_a']+df.loc[t,'acc_clicks_b']))
            writer.grab_frame(facecolor=fig.get_facecolor(), edgecolor='none')
    plt.close()


    # A/B-test - on campaign CpC data...
    writer, fig, plt, plts, txt_time = make_video(xlabel='Cost-per-Click (AB-test)', n_versions=2, colormap=['#ff4444','#ff44ff'], txt_pos=0.4)
    plt.xlim([0, 0.5])
    plt.ylim([0, 20])
    with writer.saving(fig, f'{VIDEO_FOLDER}/AB-test_CpC.mp4', 100):
        for t in tqdm(range(N_STEPS+1)):
            plts[0].set_data(x, stats.gamma.pdf(x, a=df.loc[t,'a_a'], scale=df.loc[t,'scale_a']))
            plts[1].set_data(x, stats.gamma.pdf(x, a=df.loc[t,'a_b'], scale=df.loc[t,'scale_b']))
            
            txt = ' time: {}\n\n A / B\n clicks: {} / {}\n cost: {:.2f} / {:.2f}\n\n'.format(t, df.loc[t,'acc_clicks_a'], df.loc[t,'acc_clicks_b'], df.loc[t,'acc_cost_a'], df.loc[t,'acc_cost_b'])
            txt += ' p-value: {:.1f}%\n\n'.format(100*df.loc[t,'pvalue_cpc'])
            txt += ' p(a<b): {:.1f}% / {:.1f}%\n'.format(100*(1-df.loc[t,'P_AB_g']), 100*(1-df.loc[t,'P_BA_g']))
            txt += ' loss: {:.3f} / {:.3f} \n\n'.format(df.loc[t,'loss_cpc_a'], df.loc[t,'loss_cpc_b'])
            txt_time.set_text(txt)
            
            writer.grab_frame(facecolor=fig.get_facecolor(), edgecolor='none')
    plt.close()